In [ ]:
import numpy as np
import pandas as pd
import warnings
import datetime as dt
import os

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

# from pandas.core.common import SettingWithCopyWarning
# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# global dict and functions
# line_style_dict = {
#     '国网电动': '-',
#     '南网电动': '--',
#     '特来电': '-',
#     '星星充电': '-',
#     '阿维塔': '-',
# }

# def get_line_style(k):
#     return line_style_dict.get(k, '--')

color_style_dict = {
    '国网电动': '#0D8C8F',
    '南网电动': '#00367A',
    '特来电': '#00A6CA',
    '星星充电': '#F0830C',
    '阿维塔': '#000000',
}

def get_color_style(key):
    return color_style_dict.get(key, '#000000')

def get_weekend_index(pd_date_range):
    res = []
    for idx, d in enumerate(pd_date_range):
        if d.weekday() in (5, 6):
            res.append(idx)
    return res

# files and date range
toc_order_file = r'D:\充电桩_战略_公共充电订单(C端)_20230313.xlsx'
FROM_DATE = dt.datetime.strptime('2023-03-01', '%Y-%m-%d')
TO_DATE = dt.datetime.strptime('2023-03-12', '%Y-%m-%d')

pd_date_range = pd.date_range(start=FROM_DATE, end=TO_DATE, freq='D')
weekend_index = get_weekend_index(pd_date_range)

output_dir = r'D:\自动生成报表\%s' % dt.date.today()
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# toc order define
toc_order_column_names = [
    "充电订单号",
    "充电完成状态",
    "用户编码",
    "用户昵称",
    "手机号【加密】",
    "运营商名称",
    "充电站名称",
    "充电枪编号",
    "订单电量(度)",
    "订单总金额(元)",
    "订单电费(元)",
    "订单服务费(元)",
    "实收金额",
    "点数卡券id",
    "点数卡对外展示名称",
    "点数卡名称",
    "点数卡抵扣金额（元）",
    "点数卡抵扣点数",
    "异常代码/原因",
    "订单创建日期",
    "充电完成日期",
    "支付完成日期",
    "订单创建时间",
    "充电完成时间",
    "支付完成时间",
    "运营商id",
    "充电站id",
]

toc_order_data_types = dict.fromkeys(toc_order_column_names, np.str0)
toc_order_data_types["订单电量(度)"] = np.float32
toc_order_data_types["订单总金额(元)"] = np.float32
toc_order_data_types["订单服务费(元)"] = np.float32
toc_order_data_types["订单电费(元)"] = np.float32
toc_order_data_types["实收金额"] = np.float32
toc_order_data_types["点数卡抵扣金额（元）"] = np.float32
toc_order_data_types["点数卡抵扣点数"] = np.float32
toc_order_data_types["订单创建日期"] = np.datetime64
toc_order_data_types["充电完成日期"] = np.datetime64
toc_order_data_types["支付完成日期"] = np.datetime64
toc_order_data_types["订单创建时间"] = np.datetime64
toc_order_data_types["充电完成时间"] = np.datetime64
toc_order_data_types["支付完成时间"] = np.datetime64

In [ ]:
# 读取源数据，预处理
df_toc_order = pd.read_excel(toc_order_file, names=toc_order_column_names, dtype=toc_order_data_types, engine='openpyxl')

# nan check
# df_toc_order.isna().any()

# nan字段补零
df_toc_order["实收金额"].fillna(0, inplace=True)
df_toc_order["点数卡抵扣金额（元）"].fillna(0, inplace=True)
df_toc_order["点数卡抵扣点数"].fillna(0, inplace=True)

# 去除电量为零
df_toc_order.drop(index=df_toc_order[df_toc_order["订单电量(度)"] == 0].index, inplace=True)

# df_toc_order.describe()

# 分析数据的时间范围，[FROM_DATE, TO_DATE]
df_target = df_toc_order[(df_toc_order["充电完成日期"] >= FROM_DATE) & (df_toc_order["充电完成日期"] <= TO_DATE)]

In [ ]:
# 订单电量及总金额频度分布
df_copy = df_target[["订单电量(度)", "订单总金额(元)"]].copy()
df_copy.loc[:, "度电单价(元)"] = df_copy.loc[:, "订单总金额(元)"] / df_copy.loc[:, "订单电量(度)"]

plt.figure(figsize=(7 * 3, 6))

ax = plt.subplot(1, 3, 1)
ax.hist(df_copy["订单电量(度)"], bins=50)
mean = df_copy["订单电量(度)"].mean()
ax.axvline(x=mean, color='green', linestyle='--')
ax.set_xlabel("订单电量(度)")
ax.set_ylabel("累计数(个)")
ax.set_title("订单电量分布\nmean = %.2f元" % mean)

ax = plt.subplot(1, 3, 2)
ax.hist(df_copy["订单总金额(元)"], bins=50)
mean = df_copy["订单总金额(元)"].mean()
ax.axvline(x=mean, color='green', linestyle='--')
ax.set_xlabel("订单总金额(元)")
ax.set_ylabel("累计数(个)")
ax.set_title("订单总金额分布\nmean = %.2f元" % mean)

ax = plt.subplot(1, 3, 3)
ax.hist(df_copy["度电单价(元)"], bins=50)
mean = df_copy["度电单价(元)"].mean()
ax.axvline(x=mean, color='green', linestyle='--')
ax.set_xlabel("度电单价(元)")
ax.set_ylabel("累计数(个)")
ax.set_title("度电单价分布\nmean = %.3f元" % mean)

plt.savefig(os.path.join(output_dir, '11.电量及金额分布.png'), dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# 按【充电完成日期】分组统计
groups = df_target.groupby("充电完成日期", as_index=False)

# 电费/服务费
df_groups = groups[[
    "订单总金额(元)", 
    "订单电费(元)", 
    "订单服务费(元)", 
    ]].sum()

# 补空缺日期
idx = pd.Index(pd_date_range, name="充电完成日期")
data = df_groups.set_index("充电完成日期").reindex(idx, fill_value=0).reset_index()

_, ax = plt.subplots(figsize=(20, 4))
ax.plot(data["充电完成日期"], data["订单总金额(元)"], label='订单总金额(元)')
ax.plot(data["充电完成日期"], data["订单电费(元)"], label='订单电费(元)')
ax.plot(data["充电完成日期"], data["订单服务费(元)"], label='订单服务费(元)')
ax.set_xlabel("订单完成日期")
ax.set_ylabel("金额(元)")
ax.set_xticks(pd_date_range)
for idx in weekend_index:
    ax.get_xticklabels()[idx].set_color("grey")
ax.set_title('电费/服务费分析(%s - %s)' % (FROM_DATE.date(), TO_DATE.date()))
ax.legend()
plt.savefig(os.path.join(output_dir, '21.总金额-电费-服务费.png'), dpi=200, bbox_inches='tight')
plt.show()

# 实收/权益抵扣
df_groups = groups[[
    "订单总金额(元)", 
    "实收金额",
    "点数卡抵扣金额（元）", 
    ]].sum()

# 补空缺日期
idx = pd.Index(pd_date_range, name="充电完成日期")
data = df_groups.set_index("充电完成日期").reindex(idx, fill_value=0).reset_index()

_, ax = plt.subplots(figsize=(20, 4))
ax.plot(data["充电完成日期"], data["订单总金额(元)"], label='订单总金额(元)')
ax.plot(data["充电完成日期"], data["实收金额"], label='实收金额(元)')
ax.plot(data["充电完成日期"], data["点数卡抵扣金额（元）"], label='点数卡抵扣金额(元)')
ax.set_xlabel("订单完成日期")
ax.set_ylabel("金额(元)")
ax.set_xticks(pd_date_range)
for idx in weekend_index:
    ax.get_xticklabels()[idx].set_color("grey")
ax.set_title('实收/权益抵扣分析(%s - %s)' % (FROM_DATE.date(), TO_DATE.date()))
ax.legend()
plt.savefig(os.path.join(output_dir, '22.总金额-实收-抵扣.png'), dpi=200, bbox_inches='tight')
plt.show()

# 度电/抵扣点数
df_groups = groups[[
    "订单电量(度)", 
    "点数卡抵扣点数",
    ]].sum()

# 补空缺日期
idx = pd.Index(pd_date_range, name="充电完成日期")
data = df_groups.set_index("充电完成日期").reindex(idx, fill_value=0).reset_index()

_, ax = plt.subplots(figsize=(20, 4))
ax.plot(data["充电完成日期"], data["订单电量(度)"], label='订单电量(度)')
ax.plot(data["充电完成日期"], data["点数卡抵扣点数"], label='点数卡抵扣点数(度)')
ax.set_xlabel("订单完成日期")
ax.set_ylabel("电量(度)")
ax.set_xticks(pd_date_range)
for idx in weekend_index:
    ax.get_xticklabels()[idx].set_color("grey")
ax.set_title('总度数/权益抵扣分析(%s - %s)' % (FROM_DATE.date(), TO_DATE.date()))
ax.legend(loc='best')
plt.savefig(os.path.join(output_dir, '23.总度数-权益抵扣.png'), dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# 按【运营商名称】、【充电完成日期】分组统计
groups = df_target.groupby(["运营商名称", "充电完成日期"], as_index=False)

# 运营商使用倾向
df_groups = groups[[
    "订单电量(度)", 
    # "点数卡抵扣点数",
    ]].sum()

# 补空缺日期
mux = pd.MultiIndex.from_product([df_groups["运营商名称"].unique(), pd.date_range(start=FROM_DATE, end=TO_DATE, freq='D')], names=["运营商名称", "充电完成日期"])
data = df_groups.set_index(["运营商名称", "充电完成日期"]).reindex(mux, fill_value=0).reset_index()

_, ax = plt.subplots(figsize=(20, 4))
tmp = np.zeros(shape=((TO_DATE - FROM_DATE).days + 1))
for k in df_groups["运营商名称"].unique():
    d = data[data["运营商名称"] == k]
    ax.bar(d["充电完成日期"], d["订单电量(度)"], label=k, bottom=tmp, color=[get_color_style(k)])
    tmp += d["订单电量(度)"].to_numpy()
ax.set_xlabel('充电完成日期')
ax.set_ylabel('订单电量(度)')
ax.set_xticks(pd.date_range(start=FROM_DATE, end=TO_DATE, freq='D'))

for idx in weekend_index:
    ax.get_xticklabels()[idx].set_color("grey")

ax.set_title('用户充电偏好分析（运营商）(%s - %s)' % (FROM_DATE.date(), TO_DATE.date()))
ax.legend(loc='best')
plt.savefig(os.path.join(output_dir, '31.用户充电偏好（运营商）.png'), dpi=200, bbox_inches='tight')
plt.show()